<p style="font-size:300%; background-color:pink; color:blue; text-align:center;line-height : 80px; margin : 0; padding : 0;">
Detector/Contador de votos</p>
<p style="font-size:240%; background-color:pink; color:red; text-align:center;line-height : 60px; margin : 0; padding : 0;">
Técnicas de Inteligencia Artificial - Visión de Máquina</p1>

<p style="font-size:200%; text-align:center; line-height : 40px;  margin-top : 0; margin-bottom : 0; "> <br>Jose Eduardo Garnica Aza - Johan Sebastian Suarez Sepulveda</p>
<p style="font-size:160%; text-align:center; line-height : 20px; margin-top : 0; "> email: jgarnicaa@unal.edu.co email: jssuarezse@unal.edu.co</p>
<p style="font-size:180%; text-align:center; line-height : 30px;  margin-top : 0; margin-bottom : 0; "> <br><br>INGENIERÍA MECATRÓNICA</p>
<p style="font-size:180%; text-align:center; line-height : 30px; margin-top : 0; "> Facultad de Ingeniería</p>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> Universidad Nacional de Colombia Sede Bogotá</p>
<br>
<img src="Integrantes.png",width=10,height=10>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> <br>4 de mayo de 2022</p>

# Proyecto detector de votos
    - El objetivo principal de este proyecto es realizar un contador de votos que pueda detectar a qué candidato fue el voto para impedir el fraude y tener una herramienta informatica de apoyo

## 1° Preproceso de imagenes
- Se cargan un número N de imagenes a las cuales se trataran para encontrar posteriormente las caracteristicas

In [7]:
#librerias
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap
import numpy as np 
import math as m
import time
import timeit as tm
import cv2
import pandas as pd
from pylab import plot,subplot,axis,stem,show,figure
from numpy import mean,cov,double,cumsum,dot,linalg,array
import os
from scipy import signal
from scipy import misc
import imutils
from sklearn import neighbors, datasets

In [8]:
#codigo para importar grupo de imagenes

inputpath="InputImages"
inputnames= os.listdir(inputpath)
print(inputnames)

outputpath="OutputImages"
if not os.path.exists(outputpath):
    os.makedirs(outputpath)
    print("Directorio de salida creado")

sift = cv2.SIFT_create()

['Blanco - copia (2).jpg', 'Blanco - copia (3).jpg', 'Blanco - copia (4).jpg', 'Blanco - copia (5).jpg', 'Blanco - copia (6).jpg', 'Blanco - copia.jpg', 'Blanco.jpg', 'Enrique Gomez - copia (2).jpg', 'Enrique Gomez - copia (3).jpg', 'Enrique Gomez - copia (4).jpg', 'Enrique Gomez - copia (5).jpg', 'Enrique Gomez - copia.jpg', 'Enrique Gomez.jpg', 'Fajardo - copia (2).jpg', 'Fajardo - copia (3).jpg', 'Fajardo - copia (4).jpg', 'Fajardo - copia (5).jpg', 'Fajardo - copia (6).jpg', 'Fajardo - copia (7).jpg', 'Fajardo - copia (8).jpg', 'Fajardo - copia (9).jpg', 'Fajardo - copia.jpg', 'Fajardo.jpg', 'Fico - copia (10).jpg', 'Fico - copia (11).jpg', 'Fico - copia (12).jpg', 'Fico - copia (2).jpg', 'Fico - copia (3).jpg', 'Fico - copia (4).jpg', 'Fico - copia (5).jpg', 'Fico - copia (6).jpg', 'Fico - copia (7).jpg', 'Fico - copia (8).jpg', 'Fico - copia (9).jpg', 'Fico - copia.jpg', 'Fico.jpg', 'Ingrid - copia (2).jpg', 'Ingrid - copia (3).jpg', 'Ingrid - copia.jpg', 'Ingrid.jpg', 'Luis Pere

In [9]:
#codigo para lectura y proceso de imagenes (canny) así como extracción de caracteristicas
#Se hace dentro de un ciclo para operar un numero de imagenes cualquiera
#trabajar siempre dentro del for para operar todas las imagenes al tiempo

#Se comentan imagenes de salida para disminuir tiempo de ejecución

#Cada columna de la matriz representa una imagen y cada fila una caracteristica

Matriz=np.zeros((len(inputnames),2))


count =0
Kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
Kernel1=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
Kernel2=cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
for inputnames in inputnames:

    #---------------------Pre proceso de imagenes--------------------------------#
    imagepath=inputpath+"/"+inputnames
    image=cv2.imread(imagepath)
    width, height = image.shape[:2]
    img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
    gris=cv2.cvtColor(img2,cv2.COLOR_RGB2GRAY)
    equ=cv2.equalizeHist(gris)
    cierre=cv2.erode(equ,Kernel2,iterations=1)
    Bordes=cv2.Canny(cierre,10,300)
    umbr=cv2.adaptiveThreshold(gris,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,12)
    dilatacion=cv2.dilate(umbr,Kernel2,iterations=1)
    gradiente=cv2.morphologyEx(umbr,cv2.MORPH_GRADIENT,Kernel2)
    img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
    contours,hierachy=cv2.findContours(umbr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
    ht, wt = img2.shape[:2]
    AreaImg=ht*wt
    mask=np.ones((ht,wt),np.uint8)*0#Solo es para ver los cortonos
    contours1,hierachy1=cv2.findContours(umbr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #Contours1 es borde externo, contours es borde de rectangulos
    cv2.drawContours(img2 ,contours,-1, (0,0,255) , 2)

    result=cv2.drawContours(mask ,contours,-1, 255 , 2)
    result1=cv2.drawContours(mask.copy() ,contours1,-1, 255 , 2)

    ret, th = cv2.threshold(result, 100, 255, cv2.THRESH_BINARY_INV)
    mask = np.zeros((ht+2, wt+2), np.uint8)
    cv2.floodFill(th, mask,(0,0),255); # position = (200,200)
    out = cv2.bitwise_not(th)
    kernel = np.ones((7,7),np.uint8)
    out=cv2.morphologyEx(out,cv2.MORPH_OPEN,cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
    #out=cv2.morphologyEx(out,cv2.MORPH_DILATE,cv2.getStructuringElement(cv2.MORPH_RECT,(5,5)))
    #out= cv2.dilate(out,kernel,iterations = 5)
    #out=cv2.bitwise_not(out)
    outres = cv2.resize(out, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )

    cnt, h = cv2.findContours(out,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    imageres=cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
    rectangulos=[]
    for i in range(len(cnt)):
                area = cv2.contourArea(cnt[i])
                if(area>(AreaImg/18) and area<(AreaImg)):
                    mask1 = np.zeros_like(img2)
                    cv2.drawContours(mask1, cnt, i, 255,-1)
                    x,y,w,h = cv2.boundingRect(cnt[i])
                    rec=[x,y,w,h]
                    rectangulos.append(rec)
                    rectangulos.sort(key=lambda rectangulo: rectangulo[0])
                    crop= imageres[ y:h+y,x:w+x]
                    #cv2.imshow("snip",crop)
                    #if(cv2.waitKey(0))==27:break                
    # cv2.destroyAllWindows()

    #Se separan los rectangulos para ordenarlos, anteriormente se ordenaron de menor a mayor en x encontrando los que están más a la izquierda y ahora en se
    #se centrará en encontrar de abajo hacía arriba
    #ESTE ORDEN SOLO SIRVE PARA TARJETON CON 9 POSIBILIDADES
    izquierda=rectangulos[0:3]
    medio=rectangulos[3:6]
    derecha=rectangulos[6:9]

    izquierda.sort(key=lambda rectangulo: rectangulo[1])
    medio.sort(key=lambda rectangulo: rectangulo[1])
    derecha.sort(key=lambda rectangulo: rectangulo[1])

    candidato=[]
    for v in range(len(izquierda)):
        x=izquierda[v][0]
        y=izquierda[v][1]
        w=izquierda[v][2]
        h=izquierda[v][3]
        crop=imageres[y:h+y,x:w+x]
        candidato.append(crop)
        x=medio[v][0]     #FUNCIONA PORQUE IZAUEIRDA MEDIO Y DERECHA TIENEN LA MISMA LONGITUD LEE EL TARJETON DE IZQUIERDA A DERECHA DE ARRIBA ABAJO
        y=medio[v][1]
        w=medio[v][2]
        h=medio[v][3]
        crop=imageres[y:h+y,x:w+x]
        candidato.append(crop)
        x=derecha[v][0]
        y=derecha[v][1]
        w=derecha[v][2]
        h=derecha[v][3]
        crop=imageres[y:h+y,x:w+x]
        candidato.append(crop)

    #cv2.imshow('CANDIDATO',candidato[2])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    #Extraccion de características
    ##Deteccion de la característica final
    t=0
    contador=0
    numerocandi=10
    cruces=0
    for contorno in candidato:
        #print("el numero del candidato es el " + str(t))
        prueba =candidato[t]
        prueba1=cv2.cvtColor(prueba,cv2.COLOR_RGB2GRAY)
        salida=cv2.adaptiveThreshold(prueba1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,17)
        salida=cv2.dilate(salida,Kernel2,iterations=2)
        final= prueba.copy()
        lines = cv2.HoughLines(salida,1,np.pi/180,200)
        y=False
        if lines is None:
            y= False
        else:   
            for line in lines:
                rho,theta = line[0]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                if(x2==x1):
                    y=False
                else:
                    m=(y2-y1)/(x2-x1)
                    m= abs(m)
                if m<2:
                    y=True
                cv2.line(final,(x1,y1),(x2,y2),(0,0,255),2)

        """if y==True and numerocandi==0:
            cruces+=1
            print("el voto es para el candidato: "+str(t))
            numerocandi=t
        elif y==True and (cruces>0):
            cruces+=1
            print("el voto es nulo")
            numerocandi=10
        elif cruces==0 and t==9:
            print("el voto está vacío")
            numerocandi=0"""

        if y==True and numerocandi==10:
            cruces+=1
            print("el voto es por: "+str(t))
            numerocandi=t
            y=False
        if y==True and cruces>0:
            cruces+=1
            print("el voto es nulo")
            numerocandi=t
            y=False
        if numerocandi!=10 and t==9:
            print("el voto está vacío")
            numerocandi=10
        
        t=t+1        
   
 #----Matriz caracteristicas-----#
    if cruces>2:
        cruces=2
        numerocandi=9
    Matriz[count][0]=numerocandi
    Matriz[count][1]=cruces
    count+=1
    


el voto es por: 8
el voto es por: 8
el voto es por: 8
el voto es por: 8
el voto es por: 8
el voto es por: 8
el voto es por: 8
el voto es por: 4
el voto es por: 4
el voto es por: 4
el voto es por: 4
el voto es por: 4
el voto es por: 4
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 3
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 2
el voto es por: 7
el voto es por: 7
el voto es por: 7
el voto es por: 7
el voto es por: 6
el voto es por: 6
el voto es por: 6
el voto es por: 6
el voto es por: 6
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es por: 1
el voto es

## Clasificadores

In [57]:
#Clasificador NN
samples=Matriz
responses=np.transpose(Matriz)
responses=responses[0]

samples = np.float32(samples)
responses = np.float32(responses)

print ("samples")
print (samples)
print ("responses")
print (responses)

Clasificador_KNN = cv2.ml.KNearest_create()

#train
Clasificador_KNN.train(samples, cv2.ml.ROW_SAMPLE, responses)

#predict(self, samples):
retval, results, neigh_resp, dists = Clasificador_KNN.findNearest(samples, k = 1)

print ("Results")
print (results.ravel())



samples
[[ 8.  1.]
 [ 8.  1.]
 [ 8.  1.]
 [ 8.  1.]
 [ 8.  1.]
 [ 8.  1.]
 [ 8.  1.]
 [ 4.  1.]
 [ 4.  1.]
 [ 4.  1.]
 [ 4.  1.]
 [ 4.  1.]
 [ 4.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 3.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 2.  1.]
 [ 7.  1.]
 [ 7.  1.]
 [ 7.  1.]
 [ 7.  1.]
 [ 6.  1.]
 [ 6.  1.]
 [ 6.  1.]
 [ 6.  1.]
 [ 6.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 5.  1.]
 [ 5.  1.]
 [ 5.  1.]
 [ 5.  1.]
 [ 5.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [10.  0.]
 [ 9.  2.]]
responses
[ 8.  8.  8.  8.  8.  8.  8.  4.  4.  4.  4.  4.  4.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  7.  7.  7.  7.  6.  6.  6.  6.  6.  1.  1

In [62]:
#Contar los votos de manera "manual"
def buscar_n_elemento(lista, e):
   contador=0
   for i in lista:
      if i == e:
         contador+=1
   return contador


print("Candidato 0: "+str(buscar_n_elemento(results, 0))+" votos")
print("Candidato 1: "+str(buscar_n_elemento(results, 1))+" votos")
print("Candidato 2: "+str(buscar_n_elemento(results, 2))+" votos")
print("Candidato 3: "+str(buscar_n_elemento(results, 3))+" votos")
print("Candidato 4: "+str(buscar_n_elemento(results, 4))+" votos")
print("Candidato 5: "+str(buscar_n_elemento(results, 5))+" votos")
print("Candidato 6: "+str(buscar_n_elemento(results, 6))+" votos")
print("Candidato 7: "+str(buscar_n_elemento(results, 7))+" votos")
print("Candidato 8: "+str(buscar_n_elemento(results, 8))+" votos")
print("Votos nulos: "+str(buscar_n_elemento(results, 9))+" votos")
print("Votos vacíos: "+str(buscar_n_elemento(results, 10))+" votos")

Candidato 0: 5 votos
Candidato 1: 15 votos
Candidato 2: 13 votos
Candidato 3: 10 votos
Candidato 4: 6 votos
Candidato 5: 5 votos
Candidato 6: 5 votos
Candidato 7: 4 votos
Candidato 8: 7 votos
Votos nulos: 1 votos
Votos vacíos: 1 votos


In [81]:
#Kmedias

samples = np.float32(samples)
# Define criteria = ( type, max_iter = 10 , epsilon = 1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
# Set flags (Just to avoid line break in the code)
flags = cv2.KMEANS_RANDOM_CENTERS
# Apply KMeans
compactness,labels,centers = cv2.kmeans(samples,11,None,criteria,20,flags)
print ("responses")
print (responses)
print ("Results")
print (np.transpose(labels))

responses
[ 8.  8.  8.  8.  8.  8.  8.  4.  4.  4.  4.  4.  4.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  7.  7.  7.  7.  6.  6.  6.  6.  6.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  5.  5.  5.  5.  5.  0.  0.  0.  0.  0. 10.  9.]
Results
[[ 3  3  3  3  3  3  3  2  2  2  2  2  2  7  7  7  7  7  7  7  7  7  7  1
   1  1  1  1  1  1  1  1  1  1  1  1  9  9  9  9  8  8  8  8  8  4  4  4
   4  4  4  4  4  4  4  4  4  4  6  5  8  8  8  8  8  0  0  0  0  0 10  3]]


In [82]:
#Impresion manual de resultados

print("Candidato 0: "+str(buscar_n_elemento(labels, 0))+" votos")
print("Candidato 1: "+str(buscar_n_elemento(labels, 1))+" votos")
print("Candidato 2: "+str(buscar_n_elemento(labels, 2))+" votos")
print("Candidato 3: "+str(buscar_n_elemento(labels, 3))+" votos")
print("Candidato 4: "+str(buscar_n_elemento(labels, 4))+" votos")
print("Candidato 5: "+str(buscar_n_elemento(labels, 5))+" votos")
print("Candidato 6: "+str(buscar_n_elemento(labels, 6))+" votos")
print("Candidato 7: "+str(buscar_n_elemento(labels, 7))+" votos")
print("Candidato 8: "+str(buscar_n_elemento(labels, 8))+" votos")
print("Votos nulos: "+str(buscar_n_elemento(labels, 9))+" votos")
print("Votos vacíos: "+str(buscar_n_elemento(labels, 10))+" votos")

Candidato 0: 5 votos
Candidato 1: 13 votos
Candidato 2: 6 votos
Candidato 3: 8 votos
Candidato 4: 13 votos
Candidato 5: 1 votos
Candidato 6: 1 votos
Candidato 7: 10 votos
Candidato 8: 10 votos
Votos nulos: 4 votos
Votos vacíos: 1 votos


In [84]:
print("Candidato 0: "+str(buscar_n_elemento(responses, 0))+" votos")
print("Candidato 1: "+str(buscar_n_elemento(responses, 1))+" votos")
print("Candidato 2: "+str(buscar_n_elemento(responses, 2))+" votos")
print("Candidato 3: "+str(buscar_n_elemento(responses, 3))+" votos")
print("Candidato 4: "+str(buscar_n_elemento(responses, 4))+" votos")
print("Candidato 5: "+str(buscar_n_elemento(responses, 5))+" votos")
print("Candidato 6: "+str(buscar_n_elemento(responses, 6))+" votos")
print("Candidato 7: "+str(buscar_n_elemento(responses, 7))+" votos")
print("Candidato 8: "+str(buscar_n_elemento(responses, 8))+" votos")
print("Votos nulos: "+str(buscar_n_elemento(responses, 9))+" votos")
print("Votos vacíos: "+str(buscar_n_elemento(responses, 10))+" votos")

Candidato 0: 5 votos
Candidato 1: 15 votos
Candidato 2: 13 votos
Candidato 3: 10 votos
Candidato 4: 6 votos
Candidato 5: 5 votos
Candidato 6: 5 votos
Candidato 7: 4 votos
Candidato 8: 7 votos
Votos nulos: 1 votos
Votos vacíos: 1 votos
